In [1]:
import requests
from bs4 import BeautifulSoup as BS
import re
import pandas as pd
import threading
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import json
import pickle
from tqdm import tqdm
import time

In [2]:
start, end = 1, 1 # 시작 페이지부터 종료 페이지 설정
page_range = [i for i in range(start, end + 1)]

In [3]:
def get_chrome_driver():
    # 1. 브라우저 옵션 세팅
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('headless') # 브라우저가 뜨지 않고 실행

    # 2. driver 생성
    driver = webdriver.Chrome(
        service = Service(ChromeDriverManager().install()),
        options = chrome_options
    )
    return driver

In [4]:
def detail(url, productNo, page_size, product): # detail url과 productNo, 가져올 댓글의 개수를 파라미터로 받음
    driver = get_chrome_driver()
    driver.get(url)
    page = driver.page_source
    soup = BS(page, "html.parser")

    try:
        title = soup.select_one("#content > div.end_head > h2").text.replace("\n"," ").strip()
    except:
        title = None
    try:
        star_score = float(soup.select_one("#content > div.end_head > div.score_area > em").text.replace("\n"," ").replace(" ", ""))
    except:
        star_score = None
    try:
        liked = int(soup.select_one("#content > div.end_head > div.user_action_area > ul > li:nth-child(2) > div > a > em").text.replace("\n"," ").replace(" ", ""))
    except:
        liked = None
    try:
        author = soup.select_one("#content > ul.end_info.NE\=a\:nvi > li > ul > li:nth-child(3) > a").text.replace("\n"," ").strip()
    except:
        author = None
    try:
        publisher = soup.select_one("#content > ul.end_info.NE\=a\:nvi > li > ul > li:nth-child(4) > a").text.replace("\n"," ").strip()
    except:
        publisher = None
    try:
        age_rating = soup.select_one("#content > ul.end_info.NE\=a\:nvi > li > ul > li:nth-child(5)").text.replace("\n"," ").strip()
    except:
        age_rating = None
    try:
        rule = re.compile("[0-9]+_[0_9]")
        sObjectId = re.findall(rule, str(soup)[str(soup).find("sObjectId"):str(soup).find("sObjectId")+22])[0]
    except:
        sObjectId = None

    URL = f"https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=series5&templateId=web&pool=cbox12&_cv=20230425152100&_callback=jQuery34103220350950592199_1682763418916&lang=ko&country=&objectId={sObjectId}&categoryId=&pageSize={page_size}&indexSize=5&groupId=&listType=OBJECT&pageType=default&page=1&initialize=true&followSize=5&userType=&useAltSort=true&replyPageSize=10&_=1682763418918"
    
    header = {
        "referer": f"https://series.naver.com/novel/detail.series?productNo={productNo}", # 지우면 안됨
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36" # 지우면 안됨
    }
    
    response = requests.get(URL, headers=header)
    data = response.content
    
    jquery = data.decode(encoding='UTF-8', errors='strict')
    
    start = jquery.find('{')
    end = jquery[::-1].find("}")
    json_data = jquery[start : -end]
    
    dict_data = json.loads(json_data)

    commentList = dict_data['result']['commentList']

    comment_list = [] # 한 작품의 댓글을 담을 리스트
    for comment in commentList:
        comment_dict = {} # 한 댓글의 정보를 담을 딕셔너리
        comment_dict['contents'] = re.sub("\s{2,}", " ", comment['contents'].replace("\n", ""))
        comment_dict['sympathyCount'] = int(comment['sympathyCount'])
        comment_dict['antipathyCount'] = int(comment['antipathyCount'])
        comment_list.append(comment_dict)

    info = {} # 한 작품의 정보가 담긴 딕셔너리 만들기
    info['title'] = title
    info['star_score'] = star_score
    info['liked'] = liked
    info['author'] = author
    info['publisher'] = publisher
    info['age_rating'] = age_rating
    info['sObjectId'] = sObjectId
    info['comment_list'] = comment_list

    product[productNo] = info


dict 정보
- product
- productNo : string
    - title : string
    - star_score : float
    - liked : int
    - author : string
    - publisher : string
    - age_rating : string
    - sObjectId : string
    - comment_list : list

- comment_list
    - contents : string
    - sympathyCount : int
    - antipathyCount : int

In [5]:
threads = []
product = {}
for page in tqdm(page_range): # 총 3072 페이지 존재
    NAVER_SERIES_URL = f"https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=all&page={page}" # 페이지에 따른 작품 가져오기

    reponse = requests.get(NAVER_SERIES_URL)
    html_code = reponse.content

    soup = BS(html_code, "html.parser") # 파싱

    items = soup.select("div#content > div.lst_thum_wrap > ul.lst_list > li > div.cont.cont_v2 > h3 > a") # 작품들 가져오기

    for item in items: # 한 페이지 내의 작품들 25개씩임
        href = "https://series.naver.com" + item['href']
        productNo = re.findall(re.compile("[0-9]+"), href)[0]
        t = threading.Thread(target = detail, args = (href, productNo, 10, product))
        t.start()
        threads.append(t)
    time.sleep(1)

100%|██████████| 1/1 [00:01<00:00,  1.53s/it]


In [6]:
for thread in tqdm(threads):
    thread.join()

100%|██████████| 25/25 [00:16<00:00,  1.52it/s]


In [7]:
product

{'9440917': {'title': '히든 피스로 전설 기사',
  'star_score': 8.3,
  'liked': 1,
  'author': 'S.Captain',
  'publisher': '문피아',
  'age_rating': '전체 이용가',
  'sObjectId': '498891_0',
  'comment_list': [{'contents': '주69시간제로 성큼 다가온 이세계 빙의 위험성',
    'sympathyCount': 132,
    'antipathyCount': 12},
   {'contents': '아...유치하네...', 'sympathyCount': 51, 'antipathyCount': 7},
   {'contents': '짜증나서 댓글 쓰는데 레벨에대한 설정을좀 제대로 해애할꺼같음 처음 검술얻었을때는 실제수준이아니구 육체에대한 강함기준으로 가는것 같더니 점점갈수록 오류가 나더니.이번화에서는 기존 육체강화에 8배인데 레벨 2상승 ? 너무 수치상승이 괴랄했음 뭐 계속 이렇게.올리는게힘들다 설정에 그만큼 강하다라는설정이면 처음 세계관최강자설정이 너무 말도안댐 설정좀 자세히 생각하고 글을써주셨으면 하는바람',
    'sympathyCount': 24,
    'antipathyCount': 0},
   {'contents': '이 책의 제목은 잘못되었다.히든피스로 주인공 강탈. 이게 맞다.',
    'sympathyCount': 21,
    'antipathyCount': 0},
   {'contents': "29화 16%쯤 켈베르트가 과거 회상하는 씬에서'명성 높은 가문의 아들로 태어나 어린 시절부터 가족들의 기대를 한 몸에 받았고, 자신 역시 그 기대에 부응할 만한 모습을 보여주었다.주변 사람 모두가 입이 마르도록 그의 실력과 인품을 칭송했고, 과연 드높은 가문의 명성에 어울리는 인재라는 평이 자자했다.'라고 묘사... 아르민 어린시절이랑 맞지않아 수정이 필요할듯 싶습니다.",
    'sympathyCount'

In [8]:
with open(f'product_{start}_{end}.txt', 'wb') as f:
    pickle.dump(product, f)

In [9]:
with open(f'product_{start}_{end}.txt', 'rb') as f:
    data = pickle.load(f)

In [10]:
data

{'9440917': {'title': '히든 피스로 전설 기사',
  'star_score': 8.3,
  'liked': 1,
  'author': 'S.Captain',
  'publisher': '문피아',
  'age_rating': '전체 이용가',
  'sObjectId': '498891_0',
  'comment_list': [{'contents': '주69시간제로 성큼 다가온 이세계 빙의 위험성',
    'sympathyCount': 132,
    'antipathyCount': 12},
   {'contents': '아...유치하네...', 'sympathyCount': 51, 'antipathyCount': 7},
   {'contents': '짜증나서 댓글 쓰는데 레벨에대한 설정을좀 제대로 해애할꺼같음 처음 검술얻었을때는 실제수준이아니구 육체에대한 강함기준으로 가는것 같더니 점점갈수록 오류가 나더니.이번화에서는 기존 육체강화에 8배인데 레벨 2상승 ? 너무 수치상승이 괴랄했음 뭐 계속 이렇게.올리는게힘들다 설정에 그만큼 강하다라는설정이면 처음 세계관최강자설정이 너무 말도안댐 설정좀 자세히 생각하고 글을써주셨으면 하는바람',
    'sympathyCount': 24,
    'antipathyCount': 0},
   {'contents': '이 책의 제목은 잘못되었다.히든피스로 주인공 강탈. 이게 맞다.',
    'sympathyCount': 21,
    'antipathyCount': 0},
   {'contents': "29화 16%쯤 켈베르트가 과거 회상하는 씬에서'명성 높은 가문의 아들로 태어나 어린 시절부터 가족들의 기대를 한 몸에 받았고, 자신 역시 그 기대에 부응할 만한 모습을 보여주었다.주변 사람 모두가 입이 마르도록 그의 실력과 인품을 칭송했고, 과연 드높은 가문의 명성에 어울리는 인재라는 평이 자자했다.'라고 묘사... 아르민 어린시절이랑 맞지않아 수정이 필요할듯 싶습니다.",
    'sympathyCount'